In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 6.4MB/s 
     |████████████████████████████████| 61kB 22.7MB/s 
     |████████████████████████████████| 471kB 41.8MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536235 sha256=6a49a9df769a66c71ca5e5589e235131f3b198a8d1675fe57a3aff006c7b98a1
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=b25a4d1983d32c79d274eab7a2eee4e490d2cbdbe6ed27b71e6dfa6101f550f4
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=69ab254376acaa3b337f90e01e7c5b1e43bc80de42db51c7b376cce1c314f2f4
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
template <class T>
__device__ T add_two_vector(T x, T y){
    return (x + y);
}

extern "C"{
__global__ void add_two_vector_kernel(int nx, int *a, int *b, int *res){
    int x = threadIdx.x + blockDim.x * blockIdx.x;
    if (x < nx){
        res[x] = add_two_vector<int>(a[x], b[x]);
    }
}
}
""", no_extern_c=True)

In [ ]:
add_two_vector = module.get_function("add_two_vector_kernel")

In [ ]:
num_components = np.int32(10)
x = np.arange(num_components, dtype=np.int32)

In [ ]:
np.random.seed(123)
y = np.random.randint(0, 10, num_components, dtype=np.int32)

In [ ]:
x_gpu = gpuarray.to_gpu(x)
y_gpu = gpuarray.to_gpu(y)

In [ ]:
res_gpu = gpuarray.zeros(num_components, dtype=np.int32)

In [ ]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [ ]:
add_two_vector(num_components, x_gpu, y_gpu, res_gpu, block=threads_per_block, grid=blocks_per_grid)

In [ ]:
res_gpu.get()

array([ 2,  3,  8,  4,  7, 14, 12,  8,  8, 10], dtype=int32)

In [ ]:
x + y

array([ 2,  3,  8,  4,  7, 14, 12,  8,  8, 10], dtype=int32)